This notebook with call the necessary functions to pass a data frame to an LLM and have it make suggestions for the output:

In [87]:
#Library import for loading df
from llm_data_checker import read_df


#incase of error with file cache uncomment below command 
#importlib.reload(llm_data_checker)

data = read_df("data_test/Uncleaned_DS_jobs.csv")

#data.head()

Run this cell to check basic stats of df:

In [86]:
from llm_data_checker import df_checker

check_data = df_checker(data)


#write out the stats to external directory
with open("outputs/stats.txt", "w") as f:
    for section_name, section_value in check_data.items():
        # Write a header for this section
        f.write(f"===== {section_name} =====\n")
        
        # Write the actual data
        f.write(str(check_data))
        
        # Add spacing
        f.write("\n\n")

Anonymiser: 

Prompt builder:

Write out results from LLM: